In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
from typing import *
from dataclasses import dataclass
import shutil

import pandas as pd

import lass.train
import lass.test
import lass.datasets
from lass.log_handling import PaperTasks, LogIssues, LogLoaderArgs, LogLoader

import transformers

transformers.logging.set_verbosity_warning() # type: ignore

## Find non-empty tasks


In [3]:
loader_args = LogLoaderArgs(
    logdir="../artifacts/logs",
    tasks='paper-full',
    model_families=["BIG-G T=0"],
    model_sizes=["128b"],
    shots=[0],
    query_types=["multiple_choice"],
)

loader = LogLoader(loader_args)
data = lass.datasets.to_dataframe(loader)
nonempty_tasks = data.task.unique().tolist()
non_empty_tasks = [task for task in PaperTasks.full() if task in nonempty_tasks]
assert "ascii_word_recognition" not in non_empty_tasks

In [4]:
@dataclass
class Architecture():
    name: str
    name_short: str
    batch_size: int
    gradient_accumulation_steps: int

architecture = Architecture(
    name="microsoft/deberta-v3-base",
    name_short="deberta",
    batch_size=16,
    gradient_accumulation_steps=2,
)

MODEL_SIZES = {
    # "2m": 2098048,
    "16m": 16780288,
    "53m": 56629632,
    "125m": 134228992,
    "244m": 262161280,
    "422m": 453009408,
    "1b": 1073784832,
    "2b": 2097218560,
    "4b": 3623973888,
    "8b": 8590102528,
    "27b": 28991404032,
    "128b": 137440272384,
}

## Train An Assessor Model for All Different BIG-G Sizes

In [5]:
for size, size_precise in MODEL_SIZES.items():
    data_args = LogLoaderArgs(
        logdir="../artifacts/logs",
        tasks='paper-full',
        model_families=["BIG-G T=0"],
        model_sizes=[size],
        shots=[3],
        query_types=["multiple_choice"],
    )

    model = lass.train.train(
        data_args=data_args,
        group="scale-relation",
        split="instance",
        model_name=architecture.name,
        model_name_short=f"{architecture.name_short}-for-{size}",
        batch_size=architecture.batch_size,
        gradient_accumulation_steps=architecture.gradient_accumulation_steps,
        include_model_in_input=False,
        include_n_targets_in_input=False,
        output_dir="scaling",
        n_epochs=8,
        extra_training_args={
            "warmup_steps": 3000,
            "learning_rate": 2e-5,
        },
        # is_test_run=True,
    )

    results = {}

    # Metrics in total
    results_ = lass.test.test(
            data_args=data_args,
            split = 'instance',
            model_loc=model,
            model_name=architecture.name,
            max_sequence_length = 512,
    )
    results['_total'] = results_['metrics']
    results['_total']['count'] = len(results_['test'])

    print("Tested on everything")

    # Metrics per task
    for task in non_empty_tasks:
        task_data_args = LogLoaderArgs(**(loader_args.__dict__ | {'tasks': [task]}))
        results_ = lass.test.test(
            data_args=task_data_args,
            split = 'instance',
            model_loc=model,
            model_name=architecture.name,
            max_sequence_length = 512,
        )
        results[task] = results_['metrics']
        results[task]['count'] = len(results_['test'])

        df = pd.DataFrame.from_dict(results, orient='index')
        df.to_csv(f"scaling/{size}.csv")
        print(f"Tested on {task}")

    print("Tested on all tasks")


/home/wout/pp/lass/src/lass/pipeline.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'correct'] = df['correct'].astype(int)


{'text': 'In what follows, we provide short narratives, each of which illustrates a common proverb. \nNarrative: The anxiety about getting the injection made her unable to sleep that night, but the actual experience wasn\'t as bad as she thought it would be.\nThis narrative is a good illustration of the following proverb: Don\'t meet troubles half-way\n\nNarrative: Inmate 76853 lay on the execution gurney ready to utter his last words before lethal drugs ended his life.  He looked through the mirror into the witness room and said to his family and his victim\'s family: "I killed Victoria.  It was wrong.  I can\'t take it back.  I earned my punishment, and I accept it."  Inmate 76853 turned his head back to stare at the stark ceiling and paid his debt with his life.\nThis narrative is a good illustration of the following proverb: The wages of sin is death\n\nNarrative: "Oil change? Why would I want that Tim told the auto tech. Tim was convinced the mechanic was just trying to figure out

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


/home/wout/pp/lass/.env/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/44 [00:00<?, ?ba/s]

...

Deleting older checkpoint [scaling/deberta-for-16m-bs16*2-3sh-instance-split-10191710/checkpoint-10000] due to args.save_total_limit




Training completed. Do not forget to share your model on huggingface.co/models =)




Loading best model from scaling/deberta-for-16m-bs16*2-3sh-instance-split-10191710/checkpoint-3000 (score: 0.5677661299705505).


eval/accuracy,▅▄▆▅██▆▇█▆▆▅▆▁▁▅▃▂▂▃▁
eval/balanced_accuracy,▁█▅▇▂▆▇█▅█▇▇▇▆▇▆▆▆▆▆▆
eval/bs,▂▂▁▁▂▁▁▁▁▁▁▂▂▄▄▄▅▆▇██
eval/bs_dsc,▄▆▆▇▇█████▇▇▇▄▄▃▂▂▂▁▁
eval/bs_mcb,▁▁▁▁▂▁▁▁▁▁▁▂▂▄▄▄▅▅▇██
eval/bs_unc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/conf_absolute_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/conf_absolute_balanced_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/conf_absolute_bs,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/conf_absolute_bs_dsc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/conf_absolute_bs_mcb,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


/home/wout/pp/lass/src/lass/pipeline.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'correct'] = df['correct'].astype(int)
